In [430]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt
from scipy.spatial.distance import squareform, pdist
import datetime
import math

In [431]:
df = pd.read_csv('data\instance01.txt', header=None)
n_S = int(df_main['n_S'])
n_C = int(df_main['n_C'])
n_L = int(df_main['n_L'])
n_K = int(df_main['n_K'])
n_D = int(df_main['n_D'])
n_T = n_D * 48
B = int(df_main['B'])//30

df_main = df.iloc[:2,:]
df_main.columns = df_main.iloc[0]
df_main = df_main[1:]

df_car =  df.iloc[3:n_C+4,:]
df_car.columns = df_car.iloc[0]
df_car = df_car.dropna(axis=1)
df_car = df_car[1:]
print(df_car)

df_car_lv = df.iloc[n_C+5:n_C+n_L+6,:]
df_car_lv.columns = df_car_lv.iloc[0]
df_car_lv = df_car_lv.dropna(axis=1)
df_car_lv = df_car_lv[1:]
print(df_car_lv)

df_order = df.iloc[n_C+n_L+7:n_C+n_L+n_K+8,:]
df_order.columns = df_order.iloc[0]
df_order = df_order.dropna(axis=1)
df_order = df_order[1:]
df_order['Time span'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - pd.to_datetime(row['Pick-up time']), axis=1)
df_order['Time units'] = df_order['Time span'].apply(lambda x: math.ceil(x.total_seconds() / 1800))
first_period_start = pd.to_datetime('2023/01/01 00:00')
time_unit = pd.Timedelta('30 minutes')
df_order['Pick-up time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Pick-up time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order['Return time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order = pd.merge(df_order, df_car_lv, left_on='Level', right_on='Car level', how='left')
df_order = df_order.drop(['Car level'], axis=1)
print(df_order)

df_distance = df.iloc[n_C+n_L+n_K+9:n_C+n_L+n_K+n_S**2+10,:]
df_distance.columns = df_distance.iloc[0]
df_distance = df_distance.dropna(axis=1)
df_distance = df_distance[1:]
distance = df_distance.pivot(index='From', columns='To', values='Distance')
distance = distance.apply(pd.to_numeric, errors='coerce') // 30

print(distance)

3 Car ID Level Initial station
4      1     1               1
5      2     2               1
6      3     2               2
8  Car level Hour rate
9          1       200
10         2       300
  Order ID Level Pick-up station Return station      Pick-up time  \
0        1     1               2              1  2023/01/02 09:00   
1        2     1               1              1  2023/01/01 00:30   
2        3     1               2              1  2023/01/02 18:00   
3        4     2               1              2  2023/01/01 08:30   
4        5     2               2              1  2023/01/01 09:30   

        Return time       Time span  Time units  Pick-up time (ordinal)  \
0  2023/01/03 12:00 1 days 03:00:00          54                      66   
1  2023/01/02 10:30 1 days 10:00:00          68                       1   
2  2023/01/03 02:00 0 days 08:00:00          16                      84   
3  2023/01/02 14:30 1 days 06:00:00          60                      17   
4  2023/01/02 02:

In [432]:
eg1 = Model("eg1")

#-------- Add variables as a list ---------#
# accept_k=1 if order k is accepted; accept_k=0 o.w.
accept = []
for k in range(n_K+1):
    if (k == 0):
        accept.append(int(1))
        continue
    accept.append(eg1.addVar(lb=0, vtype=GRB.BINARY, name='accept_'+str(k+1)))

# wimt = 1 if car m is at station i in period t; wimt = 0 o.w.
w = []
for i in range(n_S):
    w.append([])
    for m in range(n_C):
        w[i].append([])
        for t in range(n_T):
            w[i][m].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "w" + str(i+1) + str(m+1) + str(t+1)))

# z_mpq = 1 if order p and order q are on car m and order p is before order q
z = []
for m in range(n_C):
    z.append([])
    for p in range(n_K+1):
        z[m].append([])
        for q in range(n_K+1):
            z[m][p].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "z_" + str(m+1) + str(p) + str(q)))

# x_mk = time spent when accept order k assigned to car m (rental time)
x = []
for m in range(n_C):
    x.append([])
    for k in range(n_K+1):
        if (k==0):
            x[m].append(int(0))
            continue
        time_units = int(df_order.loc[k-1, 'Time units'])
        x[m].append(eg1.addVar(lb = time_units, vtype = GRB.INTEGER, name = "x_" + str(m+1) + str(k)))

# if there is no level-l's car in station i       
# uk = 1 if order k is upgraded
u = []
for k in range(n_K+1):
    if (k==0):
        u.append(int(0))
        continue
    u.append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "u_" + str(k)))


# Revenue_k = revenue of order k
Revenue = []
for k in range(n_K+1):
    if (k==0):
        Revenue.append(int(0))
        continue
    time_units = int(df_order.loc[k-1, 'Time units'])
    hour_rate = int(df_order.loc[k-1, 'Hour rate'])
    Revenue.append(eg1.addVar(lb=time_units*hour_rate, vtype=GRB.INTEGER, name="Revenue_" + str(k)))

lv_Dk = []
for k in range(n_K+1):
    if (k==0):
        lv_Dk.append(int(0))
        continue    
    level_demanded = int(df_order.loc[k-1, 'Level'])
    lv_Dk.append(eg1.addVar(lb=level_demanded, vtype=GRB.INTEGER, name="lv_Dk" + str(k)))

# # lv_Sm = supplied car level of car m
# lv_Sm = []
# for m in range(n_C):
#     level_supplied = int(df_car.loc[m, 'Level'])
#     lv_Sm.append(eg1.addVar(lb=level_supplied, vtype=GRB.INTEGER, name="lv_Sm" + str(m+1)))

# re_ij = rearrangment time to move car from station i to station j
re = []
for i in range(n_S):
    re.append([])
    for j in range(n_S):
        re[i].append(eg1.addVar(lb=distance.iloc[j, i], vtype=GRB.INTEGER, name="re_" + str(i+1) + str(j+1)))

# s_mpq: setup time for car m to process order q after finish order p
s = []
for m in range(n_C):
    s.append([])
    for p in range(n_K+1):
        s[m].append([])
        for q in range(n_K+1):
            s[m][p].append(eg1.addVar(lb=0, vtype=GRB.INTEGER, name='s_'+str(m+1)+str(p)+str(q)))

# Ck = completion time of order k
C = []
for k in range(n_K+1):
    if (k==0):
        C.append(int(0))
        continue   
    C.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "C_" + str(k)))

# Om = latest completion time of orders on car m
O = []
for m in range(n_C):
    O.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "O" + str(m+1)))

Cmax = eg1.addVar(lb = 48*n_D, vtype = GRB.INTEGER, name = "Cmax")
A = eg1.addVar(lb = 10000, vtype = GRB.INTEGER, name = "a large number")

In [433]:
eg1.setObjective(
    quicksum(accept[k]*Revenue[k] - (1-accept[k])*2*Revenue[k] for k in range(n_K+1)),
    GRB.MAXIMIZE)

In [434]:
# Each order has a single predecessor (successor) on exactaly one of the car
eg1.addConstrs((quicksum(z[m][p][q] for m in range(n_C) for p in range(n_K+1) if (p!=q)) == 1) for q in range(n_K))
eg1.addConstrs((quicksum(z[m][p][q] for m in range(n_C) for q in range(n_K+1) if (p!=q)) == 1) for p in range(n_K))

# An order can only have a predecessor on a car if it also has a successor on that same car
eg1.addConstrs((quicksum(z[m][p][q] for q in range(n_K+1) if (p!=q)) == quicksum(z[m][h][p] for h in range(n_K+1) if (h!=p))) for p in range(n_K) for m in range(n_C))

# the completion times of each order s.t. if order p precedes order q on a car m, 
# the earliest time that order q can end must be greater than Cp + setup time from order p to order q and the processing time of order q
eg1.addConstrs((C[q]-C[p] + A*(1-z[m][p][q]) >= s[m][p][q] + x[m][q]) for p in range(n_K+1) for q in range(n_K) if (p!=q) for m in range(n_C))

# Only one order can be schedueled first on each car
eg1.addConstrs((quicksum(z[m][0][p] for p in range(n_K)) <= 1) for m in range(n_C))

# C0=0, an auxiliary order used to enforce the start of the schedule
eg1.addConstrs(C[0] == 0 for m in range(n_C))

# Calculate the makespan for each individual car
eg1.addConstrs((quicksum(((s[m][p][q] + x[m][q]) * z[m][p][q]) for q in range(n_K) for p in range(n_K+1) if (p!=q))  == O[m]) for m in range(n_C))
# Links the individual car makespan to the overall schedule makespan
eg1.addConstrs((O[m] <= Cmax for m in range(n_C)))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [435]:
eg1.optimize()
print("z* = ", eg1.ObjVal)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34 rows, 1129 columns and 321 nonzeros
Model fingerprint: 0xf0f5b55c
Model has 5 quadratic objective terms
Model has 78 quadratic constraints
Variable types: 0 continuous, 1129 integer (982 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [6e+00, 6e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [1e+00, 1e+00]
Presolve removed 9 rows and 938 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'ObjVal'